## 1) Importar librerias para el caso

In [86]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

## 2) Cargar dataframes

In [87]:
Afiliados_Activos = pd.read_csv("Afiliados Activos.txt", delimiter='\t')
Afiliados_Fugados = pd.read_csv("Afiliados Fugados.txt", delimiter='\t')

In [88]:
Afiliados_Activos.head(5)

,ID,Estado,Periodo,año,Género,Edad,"""IBC""",Ciudad,Origen_Afiliado,Origen_AFP,Nivel_de_Satisfacción,PQR
0,1,Activo,2017-11,2017,F,18,4066223,Bogotá,Traslado de AFP,Protección,3,No
1,2,Activo,2017-12,2017,M,65,1362789,Bogotá,Traslado de AFP,Porvenir,10,Si
2,3,Activo,2017-1,2017,F,66,1594382,Bogotá,Traslado de AFP,Protección,10,No
3,4,Activo,2017-2,2017,F,18,4794222,Bogotá,Traslado de AFP,Porvenir,10,No
4,5,Activo,2017-3,2017,F,18,4621926,Cartagena,Traslado de AFP,Porvenir,10,No


In [89]:
Afiliados_Fugados.head(5)

,ID,Estado,Periodo,Año,Género,"""Edad""","""IBC""",Ciudad,Origen_Afiliado,Origen_AFP,Nivel_de_Satisfacción,PQR
0,47773,fugado,2017-11,2017,F,40,1167421,Bogotá,Traslado de AFP,Porvenir,1,SI
1,47774,fugado,2017-12,2017,F,40,6742176,Bogotá,Traslado de AFP,Skandia,3,SI
2,47775,fugado,2017-1,2017,F,39,1300619,Bogotá,Traslado de AFP,Porvenir,2,SI
3,47776,fugado,2017-2,2017,F,31,1525194,Bogotá,Traslado de AFP,Porvenir,3,SI
4,47777,fugado,2017-3,2017,M,40,1570844,Barranquilla,Traslado de AFP,Porvenir,1,SI


## 3) Limpiar dataframes

In [90]:
# Tipo de datos primer df:
Afiliados_Activos.dtypes

ID                        int64
Estado                   object
Periodo                  object
año                       int64
Género                   object
Edad                      int64
 "IBC"                    int64
Ciudad                   object
Origen_Afiliado          object
Origen_AFP               object
Nivel_de_Satisfacción     int64
PQR                      object
dtype: object

In [91]:
# Tipo de datos segundo df:
Afiliados_Fugados.dtypes

ID                        int64
Estado                   object
Periodo                  object
Año                       int64
Género                   object
 "Edad"                   int64
 "IBC"                    int64
Ciudad                   object
Origen_Afiliado          object
Origen_AFP               object
Nivel_de_Satisfacción     int64
PQR                      object
dtype: object

In [92]:
# Ajustar tipo de dato de "Periodo" para dejarlo como fecha:

Afiliados_Activos['Periodo'] = pd.to_datetime(Afiliados_Activos['Periodo'], format='%Y-%m')
Afiliados_Fugados['Periodo'] = pd.to_datetime(Afiliados_Fugados['Periodo'], format='%Y-%m')

In [93]:
Afiliados_Fugados.dtypes

ID                                int64
Estado                           object
Periodo                  datetime64[ns]
Año                               int64
Género                           object
 "Edad"                           int64
 "IBC"                            int64
Ciudad                           object
Origen_Afiliado                  object
Origen_AFP                       object
Nivel_de_Satisfacción             int64
PQR                              object
dtype: object

In [94]:
Afiliados_Fugados.head(5)

,ID,Estado,Periodo,Año,Género,"""Edad""","""IBC""",Ciudad,Origen_Afiliado,Origen_AFP,Nivel_de_Satisfacción,PQR
0,47773,fugado,2017-11-01,2017,F,40,1167421,Bogotá,Traslado de AFP,Porvenir,1,SI
1,47774,fugado,2017-12-01,2017,F,40,6742176,Bogotá,Traslado de AFP,Skandia,3,SI
2,47775,fugado,2017-01-01,2017,F,39,1300619,Bogotá,Traslado de AFP,Porvenir,2,SI
3,47776,fugado,2017-02-01,2017,F,31,1525194,Bogotá,Traslado de AFP,Porvenir,3,SI
4,47777,fugado,2017-03-01,2017,M,40,1570844,Barranquilla,Traslado de AFP,Porvenir,1,SI


### 3.1) Añadir columna a cada df para categorizar clientes fugados de los activos:

In [95]:
Afiliados_Activos['Activo/Fugado'] = 1 # 1 representa que está activo

In [96]:
Afiliados_Fugados['Activo/Fugado'] = 2 # 2 representa que está fugado

### 3.2) Unir dataframes en uno solo

In [97]:
# Quitar espacios inservibles de los encabezados de un dataframe para poder unirlos:

Afiliados_Fugados.columns = Afiliados_Fugados.columns.str.strip(' "')
Afiliados_Activos.columns = Afiliados_Activos.columns.str.strip(' "')

In [98]:
# Quitar comillas de algunos encabezados, renombrando las columnas:

Afiliados_Fugados = Afiliados_Fugados.rename(columns={
    '"Edad"': 'Edad',
    '"IBC"': 'IBC'
})

Afiliados_Activos = Afiliados_Activos.rename(columns={
    'año': 'Año',
    '"IBC"': 'IBC'
})

In [99]:
# validar cambio y que ambos df tengan la misma estructura de nombres y orden de columnas:

print(Afiliados_Fugados.columns)
print(Afiliados_Activos.columns)

Index(['ID', 'Estado', 'Periodo', 'Año', 'Género', 'Edad', 'IBC', 'Ciudad',
       'Origen_Afiliado', 'Origen_AFP', 'Nivel_de_Satisfacción', 'PQR',
       'Activo/Fugado'],
      dtype='object')
Index(['ID', 'Estado', 'Periodo', 'Año', 'Género', 'Edad', 'IBC', 'Ciudad',
       'Origen_Afiliado', 'Origen_AFP', 'Nivel_de_Satisfacción', 'PQR',
       'Activo/Fugado'],
      dtype='object')


In [100]:
# Unir df's:

Afiliados_total = pd.concat([Afiliados_Activos, Afiliados_Fugados], ignore_index=True)
Afiliados_total.head(5)

,ID,Estado,Periodo,Año,Género,Edad,IBC,Ciudad,Origen_Afiliado,Origen_AFP,Nivel_de_Satisfacción,PQR,Activo/Fugado
0,1,Activo,2017-11-01,2017,F,18,4066223,Bogotá,Traslado de AFP,Protección,3,No,1
1,2,Activo,2017-12-01,2017,M,65,1362789,Bogotá,Traslado de AFP,Porvenir,10,Si,1
2,3,Activo,2017-01-01,2017,F,66,1594382,Bogotá,Traslado de AFP,Protección,10,No,1
3,4,Activo,2017-02-01,2017,F,18,4794222,Bogotá,Traslado de AFP,Porvenir,10,No,1
4,5,Activo,2017-03-01,2017,F,18,4621926,Cartagena,Traslado de AFP,Porvenir,10,No,1


In [101]:
Afiliados_total['Género'] = Afiliados_total['Género'].replace({
    'F': 1,
    'M': 2
})

In [102]:
Afiliados_total.head(3)

,ID,Estado,Periodo,Año,Género,Edad,IBC,Ciudad,Origen_Afiliado,Origen_AFP,Nivel_de_Satisfacción,PQR,Activo/Fugado
0,1,Activo,2017-11-01,2017,1,18,4066223,Bogotá,Traslado de AFP,Protección,3,No,1
1,2,Activo,2017-12-01,2017,2,65,1362789,Bogotá,Traslado de AFP,Porvenir,10,Si,1
2,3,Activo,2017-01-01,2017,1,66,1594382,Bogotá,Traslado de AFP,Protección,10,No,1


In [103]:
Afiliados_total.dtypes

ID                                int64
Estado                           object
Periodo                  datetime64[ns]
Año                               int64
Género                            int64
Edad                              int64
IBC                               int64
Ciudad                           object
Origen_Afiliado                  object
Origen_AFP                       object
Nivel_de_Satisfacción             int64
PQR                              object
Activo/Fugado                     int64
dtype: object

In [104]:
# Validar valores en nulo:

Afiliados_total.isnull().any().any()  # Si deseas saber si hay algún valor nulo en todo el DataFrame (no solo por columnas), puedes encadenar otro método any()

False

In [108]:
Afiliados_total['Periodo'] = Afiliados_total['Periodo'].dt.month
Afiliados_total.dtypes

ID                        int64
Estado                   object
Periodo                   int64
Año                       int64
Género                    int64
Edad                      int64
IBC                       int64
Ciudad                   object
Origen_Afiliado          object
Origen_AFP               object
Nivel_de_Satisfacción     int64
PQR                      object
Activo/Fugado             int64
dtype: object

## 4) Creación del modelo

In [105]:
# Quiero enforcarme en predecir la columna "Activo / Fugado", entonces:

X = Afiliados_total.drop('Activo/Fugado', axis=1)
y = Afiliados_total['Activo/Fugado']

# Ahora dividamos los datos en conjuntos de entrenamiento y prueba.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
